### Install lib 

In [1]:
%pip install meteostat pandas xlrd datetime


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


#### Data meteostat: Historical weather data of 8 departments in ile de france 


In [2]:
from meteostat import Point, Daily,Hourly
from datetime import datetime
import pandas as pd

In [3]:
# Define 8 coordinates
idf_points = {
    "Paris": (48.8566, 2.3522),
    "Hauts-de-Seine": (48.8924, 2.2060), # Nanterre
    "Seine-Saint-Denis": (48.9109, 2.4395), # Bobigny
    "Val-de-Marne": (48.7904, 2.4556), #Créteil
    "Yvelines": (48.8049, 2.1204), # Versailles
    "Essonne": (48.6290, 2.4410), # Evry
    "Val-d'Oise ": (49.0360, 2.0630), # Cergy
    "Seine-et-Marne": (48.5390, 2.6550) #Melun
}

# Define time range
start = datetime(2024, 1, 1)
end = datetime(2024, 12, 31)

# Retrieve daily data for each city
dfs = []
for city, (lat, lon) in idf_points.items():
    p = Point(lat, lon)
    data = Hourly(p, start, end).fetch().reset_index()
    data['city'] = city
    dfs.append(data)

weather_all = pd.concat(dfs, ignore_index=True)
weather_all



,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco,city
0,2024-01-01 00:00:00,7.1,3.9,80.0,0.0,0.0,230.0,16.6,46.0,1007.2,0.0,2.0,Paris
1,2024-01-01 01:00:00,7.4,4.2,80.0,0.0,<NA>,240.0,16.6,42.6,1007.3,0.0,2.0,Paris
2,2024-01-01 02:00:00,7.3,3.9,79.0,0.0,<NA>,230.0,18.4,42.6,1007.8,0.0,2.0,Paris
3,2024-01-01 03:00:00,7.0,3.6,79.0,0.0,0.0,230.0,20.5,31.0,1008.0,0.0,2.0,Paris
4,2024-01-01 04:00:00,7.0,3.2,77.0,0.0,<NA>,230.0,18.4,37.0,1008.5,0.0,3.0,Paris
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70083,2024-12-30 20:00:00,1.0,0.6,97.0,0.0,<NA>,210.0,11.2,16.7,1032.2,0.0,3.0,Seine-et-Marne
70084,2024-12-30 21:00:00,0.8,0.4,97.0,0.0,0.0,230.0,13.0,22.0,1032.0,0.0,3.0,Seine-et-Marne
70085,2024-12-30 22:00:00,0.8,0.5,98.0,0.0,<NA>,240.0,13.0,20.4,1032.1,0.0,3.0,Seine-et-Marne
70086,2024-12-30 23:00:00,0.8,0.5,98.0,0.0,<NA>,210.0,9.4,20.4,1032.4,0.0,5.0,Seine-et-Marne


In [4]:
weather_all.isnull().sum()

time        0
temp        0
dwpt        0
rhum        0
prcp      917
snow    51083
wdir        0
wspd        0
wpgt     8761
pres        0
tsun     8761
coco      251
city        0
dtype: int64

In [5]:
weather_all.describe()

,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
count,70088,70088.0,70088.0,70088.0,69171.0,19005.0,70088.0,70088.0,61327.0,70088.0,61327.0,69837.0
mean,2024-07-01 12:00:00,12.619331,8.716628,79.160727,0.107233,0.01768,197.08408,12.467308,23.498744,1016.214854,7.957653,3.715595
min,2024-01-01 00:00:00,-6.9,-10.7,25.0,0.0,0.0,0.0,0.0,0.0,981.0,0.0,1.0
25%,2024-04-01 06:00:00,8.3,5.4,69.0,0.0,0.0,130.0,7.6,14.8,1011.0,0.0,2.0
50%,2024-07-01 12:00:00,12.3,9.1,83.0,0.0,0.0,210.0,11.2,21.0,1016.3,0.0,3.0
75%,2024-09-30 18:00:00,16.9,12.4,92.0,0.0,0.0,270.0,16.6,29.6,1023.0,12.0,4.0
max,2024-12-31 00:00:00,37.0,23.9,100.0,20.0,7.0,360.0,63.0,91.0,1038.6,60.0,25.0
std,NaN,6.558909,5.172106,15.378182,0.50401,0.26115,96.76033,6.779357,11.591247,10.094825,12.984435,2.894431


#### Drop snow, tsun


In [6]:
feature_meteo = ["time","temp","dwpt","rhum","prcp","wdir","wspd","pres","coco","city"]


In [7]:
df_weather = weather_all[feature_meteo]

#### Data electricity ile de france

In [8]:
ele = pd.read_csv( "ile_de_france_ele.xls",sep= "\t",encoding="latin-1")  
ele.head()

,Périmètre,Nature,Date,Heures,Consommation,Thermique,Nucléaire,Eolien,Solaire,Hydraulique,...,TCH Nucléaire (%),TCO Eolien (%),TCH Eolien (%),TCO Solaire (%),TCH Solaire (%),TCO Hydraulique (%),TCH Hydraulique (%),TCO Bioénergies (%),TCH Bioénergies (%),Unnamed: 29
0,Ile-de-France,Données consolidées,2024-01-01,00:00,7843.0,197.0,0.0,126.0,0.0,1.0,...,ND,1.61,86.9,0.0,0.0,0.01,5.26,1.81,43.96,NaN
1,Ile-de-France,Données consolidées,2024-01-01,00:15,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ile-de-France,Données consolidées,2024-01-01,00:30,8043.0,197.0,0.0,126.0,0.0,1.0,...,ND,1.57,86.9,0.0,0.0,0.01,5.26,1.82,45.20,NaN
3,Ile-de-France,Données consolidées,2024-01-01,00:45,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Ile-de-France,Données consolidées,2024-01-01,01:00,7731.0,197.0,0.0,126.0,0.0,1.0,...,ND,1.63,86.9,0.0,0.0,0.01,5.26,1.86,44.58,NaN


In [9]:
ele.columns

Index(['Périmètre', 'Nature', 'Date', 'Heures', 'Consommation', 'Thermique',
       'Nucléaire', 'Eolien', 'Solaire', 'Hydraulique', 'Pompage',
       'Bioénergies', 'Stockage batterie', 'Déstockage batterie',
       'Eolien terrestre', 'Eolien offshore', 'Ech. physiques',
       'TCO Thermique (%)', 'TCH Thermique (%)', 'TCO Nucléaire (%)',
       'TCH Nucléaire (%)', 'TCO Eolien (%)', 'TCH Eolien (%)',
       'TCO Solaire (%)', 'TCH Solaire (%)', 'TCO Hydraulique (%)',
       'TCH Hydraulique (%)', 'TCO Bioénergies (%)', 'TCH Bioénergies (%)',
       'Unnamed: 29'],
      dtype='object')

In [10]:
# drop unamed: 29
feature_ele = ['Périmètre', 'Nature', 'Date', 'Heures', 'Consommation', 'Thermique',
       'Nucléaire', 'Eolien', 'Solaire', 'Hydraulique', 'Pompage',
       'Bioénergies', 'Stockage batterie', 'Déstockage batterie',
       'Eolien terrestre', 'Eolien offshore', 'Ech. physiques',
       'TCO Thermique (%)', 'TCH Thermique (%)', 'TCO Nucléaire (%)',
       'TCH Nucléaire (%)', 'TCO Eolien (%)', 'TCH Eolien (%)',
       'TCO Solaire (%)', 'TCH Solaire (%)', 'TCO Hydraulique (%)',
       'TCH Hydraulique (%)', 'TCO Bioénergies (%)', 'TCH Bioénergies (%)']
df_ele = ele[feature_ele]

In [11]:
df_ele.dropna()


,Périmètre,Nature,Date,Heures,Consommation,Thermique,Nucléaire,Eolien,Solaire,Hydraulique,...,TCO Nucléaire (%),TCH Nucléaire (%),TCO Eolien (%),TCH Eolien (%),TCO Solaire (%),TCH Solaire (%),TCO Hydraulique (%),TCH Hydraulique (%),TCO Bioénergies (%),TCH Bioénergies (%)
0,Ile-de-France,Données consolidées,2024-01-01,00:00,7843.0,197.0,0.0,126.0,0.0,1.0,...,0.0,ND,1.61,86.90,0.0,0.0,0.01,5.26,1.81,43.96
2,Ile-de-France,Données consolidées,2024-01-01,00:30,8043.0,197.0,0.0,126.0,0.0,1.0,...,0.0,ND,1.57,86.90,0.0,0.0,0.01,5.26,1.82,45.20
4,Ile-de-France,Données consolidées,2024-01-01,01:00,7731.0,197.0,0.0,126.0,0.0,1.0,...,0.0,ND,1.63,86.90,0.0,0.0,0.01,5.26,1.86,44.58
6,Ile-de-France,Données consolidées,2024-01-01,01:30,7559.0,198.0,0.0,127.0,0.0,1.0,...,0.0,ND,1.68,87.59,0.0,0.0,0.01,5.26,1.85,43.34
8,Ile-de-France,Données consolidées,2024-01-01,02:00,7415.0,199.0,0.0,124.0,0.0,1.0,...,0.0,ND,1.67,85.52,0.0,0.0,0.01,5.26,1.97,45.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35126,Ile-de-France,Données consolidées,2024-12-31,21:30,9699.0,149.0,0.0,44.0,0.0,2.0,...,0.0,ND,0.45,30.34,0.0,0.0,0.02,10.53,1.44,40.70
35128,Ile-de-France,Données consolidées,2024-12-31,22:00,9480.0,150.0,0.0,54.0,0.0,2.0,...,0.0,ND,0.57,37.24,0.0,0.0,0.02,10.53,1.47,40.41
35130,Ile-de-France,Données consolidées,2024-12-31,22:30,9599.0,150.0,0.0,67.0,0.0,2.0,...,0.0,ND,0.70,46.21,0.0,0.0,0.02,10.53,1.47,40.99
35132,Ile-de-France,Données consolidées,2024-12-31,23:00,9727.0,148.0,0.0,68.0,0.0,2.0,...,0.0,ND,0.70,46.90,0.0,0.0,0.02,10.53,1.46,41.28


In [18]:
df_ele.shape

(35137, 29)

In [13]:
ele.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35137 entries, 0 to 35136
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Périmètre            35137 non-null  object 
 1   Nature               35136 non-null  object 
 2   Date                 35136 non-null  object 
 3   Heures               35136 non-null  object 
 4   Consommation         17568 non-null  float64
 5   Thermique            17568 non-null  float64
 6   Nucléaire            17568 non-null  float64
 7   Eolien               17568 non-null  float64
 8   Solaire              17568 non-null  float64
 9   Hydraulique          17568 non-null  float64
 10  Pompage              17568 non-null  float64
 11  Bioénergies          17568 non-null  float64
 12  Stockage batterie    17568 non-null  float64
 13  Déstockage batterie  17568 non-null  float64
 14  Eolien terrestre     17568 non-null  float64
 15  Eolien offshore      17568 non-null 

In [14]:
# Rename colunms ele
ele = ele.rename(columns = {"Périmètre":"Perimeter",
                            "Nucléaire":"Nuclear",
                            "Bioénergies":"Bioenergies",
                            "Déstockage batterie ":"Battery clearance",
                            "TCO Nucléaire (%)":"TCO Nuclear (%)",
                            "TCH Nucléaire (%)":"TCH Nuclear",
                            "TCO Bioénergies (%)":"TCO Bioenergies (%)",
                            "TCH Bioénergies(%)":"TCh Bioenergies (%)"})

#### source population : https://idf.handidonnees.fr/page/population

In [15]:
population_idf = {"Paris":"2048472",
                "Hauts-de-Seine":"1663974",
                "Seine-Saint-Denis":"1711876",
                "Val-de-Marne":"1426433",
                "Yvelines":"1489363",
                "Essonne":"1343527",
                "Val-d'Osie":"1287589",
                 "Seine-et-Marne":"1479615"}
for ele,items in population_idf.items():
    print(ele,items)

Paris 2048472
Hauts-de-Seine 1663974
Seine-Saint-Denis 1711876
Val-de-Marne 1426433
Yvelines 1489363
Essonne 1343527
Val-d'Osie 1287589
Seine-et-Marne 1479615


#### In the reason of disable of data electricity in each department of ile de france, so we need to apply a method that we can obtain from an approximate of utilization of each department base on the each number population of the level aggregation in the IDF. Hence we have the method as follow: 
#### Let: <br>
&nbsp; $E_{d,t}$ &nbsp; is an utilization of the electricity in department d at the ti me t. <br>
&nbsp;   $E_{IDF,t}$ &nbsp; is an utilization of the electricity in IDF at the time t.  <br>
&nbsp;  $Pop_d$ &nbsp; is number of population in departement d. <br>

$$E_{d,t} = E_{IDF,t} \times \frac{Pop_d}{\sum_d{Pop_d}}$$